In [2]:
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import glob
import os

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import pandas as pd
pd.reset_option('display.max_rows')
print(pd.options.display.max_rows)
pd.reset_option('display.max_columns')
print(pd.options.display.max_columns)
import numpy as np

import stock_research as sr


60
20


In [3]:
EDINET_API_URL = "https://disclosure.edinet-fsa.go.jp/api/v1"
orignal_data_path = '../warehouse/stock/original_data/'
ymd_list = [i.split('\\')[-1] for i in glob.glob(orignal_data_path+'2*')]
print(min(ymd_list),max(ymd_list))
last_ym = (datetime.strptime(min(ymd_list),'%Y-%m-%d') + relativedelta(months=-1)).strftime('%Y%m')
last_ymd_first = last_ym +'01'
last_ymd_last = (datetime.strptime(last_ym,'%Y%m') + relativedelta(months=1,days=-1)).strftime('%Y%m%d')
last_ymd_first,last_ymd_last

2022-01-04 2022-04-28


('20211201', '20211231')

In [4]:
doc_type_code=['120']
SUMMARY_TYPE = 2
start_date = datetime(
    int(last_ymd_first[:4]),
    int(last_ymd_first[4:6]),
    int(last_ymd_first[-2:]))
end_date = datetime(
    int(last_ymd_last[:4]),
    int(last_ymd_last[4:6]),
    int(last_ymd_last[-2:]))
sdate = start_date.strftime('%Y%m%d')
edate = end_date.strftime('%Y%m%d')
i = -1
csv_name = f'submitted_doc_list_{sdate}_{edate}.csv'
save_path = '../warehouse/stock/created/'

In [6]:
for i, date in enumerate(sr.date_range(start_date, end_date)):
    date_str = str(date)[:10]
    df_doc_summary = sr.download_all_documents(date_str, orignal_data_path, EDINET_API_URL,
                                                   SUMMARY_TYPE, doc_type_code)
    if i == 0:
        df_doc_summary_all = df_doc_summary.copy()
    else:
        df_doc_summary_all = pd.concat([df_doc_summary_all, df_doc_summary])
    print(date_str,df_doc_summary_all.shape)


https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-01 (1, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-02 (11, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-03 (38, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-04 (38, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-05 (38, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-06 (64, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-07 (75, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-08 (98, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-09 (130, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-10 (182, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-11 (182, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
2021-12-12 (182, 27)
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json
202

In [14]:
ymd_list = [i.split('\\')[-1] for i in glob.glob(orignal_data_path+'2*')]
start_date = datetime(
    int(min(ymd_list)[:4]),
    int(min(ymd_list)[5:7]),
    int(min(ymd_list)[8:]),
    )
end_date = datetime(
    int(max(ymd_list)[:4]),
    int(max(ymd_list)[5:7]),
    int(max(ymd_list)[8:]))

In [ ]:
for date in sr.date_range(start_date, end_date):
    date_str = str(date)[:10]
    tmppath = orignal_data_path + date_str + '/doc_summary.csv'
    if os.path.exists(tmppath):
        i+=1
        df_doc_summary = pd.read_csv(orignal_data_path + date_str + '/doc_summary.csv')
        if i == 0:
            df_doc_summary_all = df_doc_summary.copy()
        else:
            df_doc_summary_all = pd.concat([df_doc_summary_all, df_doc_summary])
df_doc_summary_all.to_csv(save_path+csv_name,index=False)   
# df_doc_summary_all = pd.read_csv(save_path+csv_name)
# df_doc_summary_all.drop('Unnamed: 0',axis=1).to_csv(save_path+csv_name,index=False)        

In [ ]:
save_path = '../warehouse/stock/created/'
stock_code_list_path = '../warehouse/stock/original_data/stock_code_list/data_j(202204_last).csv'
stock_code_list = pd.read_csv(stock_code_list_path)
stock_code_list.rename(columns={'コード': 'secCode'},inplace=True)
df = pd.read_csv(save_path+csv_name)
df = df.dropna(subset=['secCode'])
df.secCode = df.secCode.astype(int)//10
df = pd.merge(df,stock_code_list,how='left')


In [77]:
csv_name = f'submitted_doc_list_only_stock_{sdate}_{edate}.csv'
df.to_csv(save_path+csv_name,index=False)

In [78]:
df = pd.read_csv(save_path+csv_name)
df.head()

,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,Unnamed: 0,日付,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,116,S100N5KL,E02762,9941,5.010001e+12,太洋物産株式会社,NaN,10,30000,120,...,NaN,20220428.0,太洋物産,スタンダード（内国株式）,6050.0,卸売業,13.0,商社・卸売,-,-
1,10,S100N7DP,E05015,4707,2.110001e+12,株式会社キタック,NaN,10,30000,120,...,NaN,20220428.0,キタック,スタンダード（内国株式）,9050.0,サービス業,10.0,情報通信・サービスその他,-,-
2,85,S100N8J8,E03336,7640,1.110001e+12,株式会社トップカルチャー,NaN,10,30000,120,...,NaN,20220428.0,トップカルチャー,スタンダード（内国株式）,6100.0,小売業,14.0,小売,7,TOPIX Small 2
3,284,S100N99O,E34814,4441,9.180001e+12,トビラシステムズ株式会社,NaN,10,30000,120,...,NaN,20220428.0,トビラシステムズ,プライム（内国株式）,5250.0,情報・通信業,10.0,情報通信・サービスその他,7,TOPIX Small 2
4,72,S100N9CX,E05376,3733,4.120001e+12,株式会社ソフトウェア・サービス,NaN,10,30000,120,...,NaN,20220428.0,ソフトウェア・サービス,スタンダード（内国株式）,5250.0,情報・通信業,10.0,情報通信・サービスその他,-,-


In [8]:
import winsound
winsound.Beep(frequency=500, duration=10000)

In [127]:

# # 文書IDと提出日を取得
# stock_code = 7425
# doc_id = df_doc_summary_all.query(f'コード=={stock_code}').docID.values[0]
# date = df_doc_summary_all.query(f'コード=={stock_code}').submitDateTime.values[0]
# file_path = orignal_data_path + date[:10] + '/' + doc_id + '/XBRL/PublicDoc/*.xbrl'
# file_path = glob.glob(file_path)[0]

# # データを取得
# sr.get_one_xbrl_data(file_path,
#               key='jpcrp_cor:NetSalesSummaryOfBusinessResults', 
#               context_ref='CurrentYearDuration')

In [128]:
# file_path = orignal_data_path + date[:10] + '/' + doc_id + '/XBRL/PublicDoc/*.xbrl'
# file_path = glob.glob(file_path)[0]
# account_df_path = '../warehouse/stock/created/account_df.csv'
# # print(f'seccode:{seccode}')
# print(f'doc_id:{doc_id}')
# print(f'date:{date}')
# print(f'file_path:{file_path}')
# print(f'account_df_path:{account_df_path}')
# account_df = pd.read_csv(account_df_path)
# sr.get_xbrl_data(file_path, account_df)

In [129]:
# # 売上を取得
# net_sales_summary = account_df.query('name=="net_sales_summary"').copy()
# # 数値データに変換
# net_sales_summary['value'] = net_sales_summary['value'].astype('float')
# # x軸用に調整
# years = [sr.calc_fiscal_year(prior_year, asof=datetime(2019,12,31))\
#          for prior_year in net_sales_summary.prior_year]

# net_sales_summary['fiscal_year'] = [str(year.year)+'/'+str(year.month) for year in years]
# # 棒グラフを作成
# plt.bar(net_sales_summary['fiscal_year'], net_sales_summary['value'], 
#         color='b', alpha=0.8, width=0.5)
# plt.title('Net Sales')

In [ ]:
# df.shape,stock_code_list.shape,pd.merge(df,stock_code_list,how='left').shape
# mask = pd.isna(pd.merge(df,stock_code_list,how='left')['銘柄名']).values
# df[mask]
# stock_code_list.query('secCode==8594')  # 銘柄コードリストはすべての銘柄を網羅しているわけではないらしい…